In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from collections import Counter
import sqlite3

In [ ]:
# Load the data
file_path = 'books.csv'
df = pd.read_csv(file_path, error_bad_lines = False)
df.head()

In [ ]:
# columns = ["bookID", "title", "authors", "average_rating", 
#              "isbn", "isbn13", "language_code", "num_pages",
#              "ratings_count", "text_reviews_count", "publication_date", "publisher"]
# target = ["recommend"]

In [ ]:
# Convert the target column values to 1 and 0 based on their values
# if average rating is less than 3.5 = 0
# if average rating is more than 3.5 = 1
df['recommend'] = np.where((df['average_rating'] >= 3.50), 1, 0) 

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df.reset_index(inplace=True, drop=True)

df.head(50)

In [ ]:
df.dtypes

split and train data

In [ ]:
# Create our features
X = pd.get_dummies(df, columns=["bookID", "title", "authors", "average_rating", 
           "isbn", "isbn13", "language_code", "  num_pages",
           "ratings_count", "text_reviews_count", "publication_date", "publisher"]).drop_first=True
#pd.get_dummies(df, drop_first=True)

In [ ]:
X = df.drop(columns="recommend")
y = df["recommend"]

In [ ]:
X.describe()

In [ ]:
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

from challenge 17: oversampling
View the count of the target classes using Counter from the collections library.
Use the resampled data to train a logistic regression model.
Calculate the balanced accuracy score from sklearn.metrics.
Print the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.

In [ ]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
## instantiate the model
ros = RandomOverSampler(random_state=1)
## resample the targets
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)

## fit
model.fit(X_resampled, y_resampled)

In [ ]:
## calculate predictions
y_pred = model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
## calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
cm_df = pd.DataFrame(
    cm, index=['Actual 1', 'Actual 0'], columns=['Predicted 1', 'Predicted 0'])
cm_df

####---##### need to double check that the results match the confusion matrix output.

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))